# COVID-19 Visualization Project
### COSC3000: Visualization, Computer Graphics & Data Analytics
#### William Kvaale

### Import dependencies

In [1]:
from os import path
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wget
pd.set_option('display.max_colwidth', -1)

### Fetch the freshest data

In [2]:
urls = [
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
    'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
]

In [3]:
def download_data(urls):
    dataset = [file for file in glob("dataset/*.csv")]
    N = len(dataset)
    loaded = True if N == 3 else False
    if not loaded:        
        for url in urls:
            print(f"Downloading csv from {url} ...")
            wget.download(url, out="dataset")
    print(f"The dataset contains {N}CSV-files:")
    print(*dataset, sep='\n')
            

In [4]:
download_data(urls)

The dataset contains 4CSV-files:
dataset/asd.csv
dataset/time_series_covid19_confirmed_global.csv
dataset/time_series_covid19_recovered_global.csv
dataset/time_series_covid19_deaths_global.csv


In [5]:
COVID_19_CONFIRMED = "dataset/time_series_covid19_confirmed_global.csv"
COVID_19_RECOVERED = "dataset/time_series_covid19_recovered_global.csv"
COVID_19_DEATHS = "dataset/time_series_covid19_deaths_global.csv"

### Load the CSV files into Pandas DataFrames

In [6]:
def load_data(filePath):
    df = pd.read_csv(filePath)
    return df

In [7]:
confirmed_df = load_data(COVID_19_CONFIRMED)
recovered_df = load_data(COVID_19_RECOVERED)
deaths_df = load_data(COVID_19_DEATHS)

In [8]:
confirmed_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,...,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,712,726,736,750,766,773,782,789,795,803
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,3256,3382,3517,3649,3848,4006,4154,4295,4474,4648
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,738,738,743,743,743,745,745,747,748,750
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,25,26,27,27,27,27,30,35,35,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
262,NaN,Sao Tome and Principe,0.186360,6.613081,0,0,0,0,0,0,...,4,4,4,8,8,14,16,16,16,23
263,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,1,1,1,1,6,6,7,10,10,12
264,NaN,Comoros,-11.645500,43.333300,0,0,0,0,0,0,...,0,0,0,0,0,1,1,3,3,3


In [9]:
dates_df = confirmed_df.columns[4:]


confirmed_latlong_df = confirmed_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    value_vars=dates_df,
    var_name='Date',
    value_name='Confirmed'
)

recovered_latlong_df = recovered_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    value_vars=dates_df,
    var_name='Date',
    value_name='Recovered'
)

deaths_latlong_df = deaths_df.melt(
    id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
    value_vars=dates_df,
    var_name='Date',
    value_name='Deaths'
)



In [10]:
 recovered_latlong_df = recovered_latlong_df[recovered_latlong_df['Country/Region']!='Canada']

In [11]:
giant_df = confirmed_latlong_df.merge(
    right=recovered_latlong_df,
    how='left',
    on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

giant_df = giant_df.merge(
    right=deaths_latlong_df,
    how='left',
    on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

In [12]:
giant_df

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Recovered,Deaths
0,NaN,Afghanistan,33.000000,65.000000,1/22/20,0,0.0,0
1,NaN,Albania,41.153300,20.168300,1/22/20,0,0.0,0
2,NaN,Algeria,28.033900,1.659600,1/22/20,0,0.0,0
3,NaN,Andorra,42.506300,1.521800,1/22/20,0,0.0,0
4,NaN,Angola,-11.202700,17.873900,1/22/20,0,0.0,0
...,...,...,...,...,...,...,...,...
27659,NaN,Western Sahara,24.215500,-12.885800,5/4/20,6,5.0,0
27660,NaN,Sao Tome and Principe,0.186360,6.613081,5/4/20,23,4.0,3
27661,NaN,Yemen,15.552727,48.516388,5/4/20,12,NaN,2
27662,NaN,Comoros,-11.645500,43.333300,5/4/20,3,0.0,0


# Preprocess and clean the data
- Convert date from string to DateTime
- Check NaN values
- Remove Cruise ships


In [13]:
giant_df['Date'] = pd.to_datetime(giant_df['Date'])

In [14]:
giant_df.isna().sum()

Province/State    19136
Country/Region    0    
Lat               0    
Long              0    
Date              0    
Confirmed         0    
Recovered         2912 
Deaths            0    
dtype: int64

Some countries report their data only by country, instead of state for state, so this makes sense.
The recovered set to NaN we will replace with 0

In [15]:
giant_df['Recovered'] = giant_df['Recovered'].fillna(0)

In [16]:

cruise_ships_df = giant_df['Province/State'].str.contains('Grand Priness') | giant_df['Province/State'].str.contains('Diamond Priness') | giant_df['Country/Region'].str.contains('Diamon Priness') | giant_df['Country/Region'].str.contains('MS Zaandam')

complete_cruise_ships_df = giant_df[~cruise_ships_df]

In [17]:
giant_df['Active'] = giant_df['Confirmed'] - giant_df['Recovered'] - giant_df['Deaths'] 

In [18]:
giant_grouped_df = giant_df.groupby(['Date', 'Country/Region'])['Confirmed', 'Recovered', 'Deaths', 'Active'].sum()

Now the indices is completely off, so we'll need to reset back to Date & Country/Region

In [19]:
giant_grouped_df = giant_grouped_df.reset_index()

### Create new columns to be placeholders for daily new updates
- New Cases
- New Recovered
- New Deaths

In [20]:
temporary_df = giant_grouped_df.groupby(['Country/Region', 'Date', ])['Confirmed', 'Recovered', 'Deaths']
temporary_df = temporary_df.sum().diff().reset_index()

masked_df = temporary_df['Country/Region'] != temporary_df['Country/Region'].shift(1)

temporary_df.loc[masked_df, 'Confirmed'] = np.nan
temporary_df.loc[masked_df, 'Recovered'] = np.nan
temporary_df.loc[masked_df, 'Deaths'] = np.nan

temporary_df.columns = ['Country/Region', 'Date', 'New Cases', 'New Recovered', 'New Deaths']



Now we have prepared the temporary new values to be merged into the complete dataframe

In [21]:
giant_grouped_df = pd.merge(giant_grouped_df, temporary_df, on=['Country/Region', 'Date'])

giant_grouped_df = giant_grouped_df.fillna(0)

giant_grouped_df[['New Cases', 'New Deaths', 'New Recovered']] = giant_grouped_df[['New Cases', 'New Recovered', 'New Deaths']].astype('int')

giant_grouped_df['New Cases'] = giant_grouped_df['New Cases'].apply(lambda c: 0 if c < 0 else c)

In [22]:
# Rename UK
giant_grouped_df['Country/Region'] = giant_grouped_df['Country/Region'].replace('United Kingdom', 'UK')

# Rename Country/Region to Country
giant_grouped_df.rename(columns={'Country/Region':'Country'}, inplace=True)


In [23]:
giant_grouped_df.sort_values(['Country', 'Date'], ascending=True, inplace=True)
giant_grouped_df

,Date,Country,Confirmed,Recovered,Deaths,Active,New Cases,New Recovered,New Deaths
0,2020-01-22,Afghanistan,0,0.0,0,0.0,0,0,0
187,2020-01-23,Afghanistan,0,0.0,0,0.0,0,0,0
374,2020-01-24,Afghanistan,0,0.0,0,0.0,0,0,0
561,2020-01-25,Afghanistan,0,0.0,0,0.0,0,0,0
748,2020-01-26,Afghanistan,0,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...
18699,2020-04-30,Zimbabwe,40,5.0,4,31.0,8,0,0
18886,2020-05-01,Zimbabwe,40,5.0,4,31.0,0,0,0
19073,2020-05-02,Zimbabwe,34,5.0,4,25.0,0,0,0
19260,2020-05-03,Zimbabwe,34,5.0,4,25.0,0,0,0


# Save to CSV

In [24]:
from datetime import datetime
date = datetime.now().strftime("%d-%m-%Y")
time = datetime.now().strftime("%H:%M")
giant_grouped_df.to_csv(f"output/COVID19-cleaned_{date}_{time}.csv")